# softmax回归的简洁实现

In [ ]:
import torch
from torch import nn
from d2l import torch as d2l
from test import load_data_fashion_mnist
from test import train_ch3

### 1. 加载小批量数据  

In [ ]:
batch_size = 256
train_iter, test_iter = load_data_fashion_mnist(batch_size)

### 2. 定义模型  
softmax回归的输出层是一个全连接层。因此，为了实现我们的模型，我们只需在Sequential中添加一个带有10个输出的全连接层。同样，在这里Sequential并不是必要的，但它是实现深度模型的基础。我们仍然以均值0和标准差0.01随机初始化权重。

In [ ]:
# PyTorch不会隐式地调整输入的形状。因此，
# 我们在线性层前定义了展平层（flatten），来调整网络输入的形状
net = nn.Sequential(nn.Flatten(), nn.Linear(784, 10))

def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal(m.weight, std = 0.01)

net.apply(init_weights)

### 3. 定义损失函数 
softmax函数$\hat y_j = \frac{\text{exp}(o_j)}{\sum_k\text{exp}(o_k)}$，其中$\hat y_j$是预测的概率分布。 $o_j$是未规范化的预测$\bold o$的第$j$个元素。如果$o_k$中的一些数值非常大，那么$\text{exp}(o_k)$可能大于数据类型容许的最大数字，即上溢（overflow）。这将使分母或分子变为inf（无穷大），最后得到的是0、inf或nan（不是数字）的$\hat y_j$。在这些情况下，我们无法得到一个明确定义的交叉熵值。   

解决这个问题的一个技巧是：在继续softmax计算之前，先从所有$o_k$中减去$\text{max}(o_k)$。  
$$
\begin{align}
\hat y_j &= \frac{\text{exp}(o_j)}{\sum_k\text{exp}(o_k)} \\
&=\frac{\text{exp}(o_j - \text{max}(o_k))\text{exp}(\text{max}(o_k))}{\sum_k \text{exp}(o_k - \text{max}(o_k))\text{exp}(\text{max}(o_k))} \\
&=\frac{\text{exp}(o_j-\text{max}(o_k))}{\sum_k \text{exp}(o_k-\text{max}(o_k))} \\
\end{align}
$$  
显然，减去$\text{max}(o_k)$，$\hat y_j$的值不变。  

在减法和规范化步骤之后，可能有些$o_j - \text{max}(o_k)$具有较大的负值。由于精度受限， $\text{exp}(o_j - \text{max}(o_k))$将有接近零的值，即下溢（underflow）。这些值可能会四舍五入为零，使$\hat y_j$为零，并且使得$\ln{\hat y_j}$的值为-inf。为了避免计算$\text{exp}(o_j - \text{max}(o_k))$，我们对交叉熵损失$\ln{\hat y_j}$进一步计算：  
$$
\begin{align}
\ln{\hat y_j} &= \ln\left(\frac{\text{exp}(o_j-\text{max}(o_k))}{\sum_k \text{exp}(o_k-\text{max}(o_k))}\right) \\
&=\ln(\text{exp}(o_j-\text{max}(o_k)))-\ln\left(\sum_k \text{exp}(o_k - \text{max}(o_k))\right) \\
&=o_j - \text{max}(o_k)-\ln\left(\sum_k \text{exp}(o_k - \text{max}(o_k))\right)
\end{align}
$$  
上述结果没有将softmax概率传递到损失函数中，而是在交叉熵损失函数中传递未规范化的预测，但我们也希望保留传统的softmax函数，以备我们需要评估通过模型输出的概率。  


In [ ]:
# nn.CrossEntropyLoss()的input和traget
# input:单个样本(C)，批样本(batch_size, C)，高维批样本(batch_size, C, d1, d2,..., dk)
# traget:类别索引or类别概率
# 类别索引：维度(batch_size)or(batch_size, d1, d2,..., dk)；值：[0, C)
# 类别概率:维度与input一致；值(0~1)
loss = nn.CrossEntropyLoss(reduction='none')

### 4. 定义优化算法  
在这里，我们使用学习率为0.1的小批量随机梯度下降作为优化算法。这与我们在线性回归例子中的相同，这说明了优化器的普适性。

In [ ]:
trainer = torch.optim.SGD(net.parameters(), lr=0.1)

### 5. 训练  

In [ ]:
num_epochs = 10
train_ch3(net, train_iter, test_iter, loss, num_epochs, trainer)